In [1]:
import logging
logging.basicConfig(level=logging.INFO)

from pygenesis import Cube

In [2]:
c = Cube(name="22922KJ1141")

In [3]:
c.get_data()

In [4]:
c.name

'22922KJ1141'

In [5]:
for k,v in c.cube.items():
    print("Cube section: ", k)
    print(v)
    print()

Cube section:  DQ
    FACH-SCHL GHH-ART GHM-WERTE-JN GENESIS-VBD REGIOSTAT EU-VBD
0  22922KJ114                    N           N         N      N

Cube section:  DQ-ERH
  FACH-SCHL
0     22922

Cube section:  DQA
     NAME RHF-BSR RHF-ACHSE
0  KREISE       1         1
1     GES       2         2
2  ERW122       3         3
3  ELGAT2       4         4

Cube section:  DQZ
   NAME ZI-RHF-BSR ZI-RHF-ACHSE
0  JAHR          5            5

Cube section:  DQI
     NAME ME-NAME   DST    TYP NKM-STELLEN GHH-ART GHM-WERTE-JN
0  ELG002     EUR  GANZ  DURCH           0                    N
1  ELG003     EUR  GANZ  DURCH           0                    N

Cube section:  QEI
      KREISE   GES      ERW122     ELGAT2  JAHR  ELG002_WERT ELG002_QUALITAET  \
0      01001  GESM  ERWERBST12  ELGPLUS-X  2016         1259                e   
1      01001  GESM  ERWERBST12  ELGPLUS-X  2017         1236                e   
2      01001  GESM  ERWERBST12  ELGPLUS-X  2018         1217                e   
3      

In [6]:
c.data

,KREISE,GES,ERW122,ELGAT2,JAHR,ELG002_WERT,ELG002_QUALITAET,ELG002_GESPERRT,ELG002_WERT-VERFAELSCHT,ELG003_WERT,ELG003_QUALITAET,ELG003_GESPERRT,ELG003_WERT-VERFAELSCHT
0,01001,GESM,ERWERBST12,ELGPLUS-X,2016,1259,e,,0,3574,e,,0
1,01001,GESM,ERWERBST12,ELGPLUS-X,2017,1236,e,,0,3898,e,,0
2,01001,GESM,ERWERBST12,ELGPLUS-X,2018,1217,e,,0,4028,e,,0
3,01001,GESM,ERWERBST12,ELGPLUS-X,2019,1299,e,,0,4045,e,,0
4,01001,GESM,ERWERBST12,ELGPLUS-X,2020,1272,e,,0,4106,e,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19180,16077,GESW,ERWERBST13,ELGPLUS,2017,173,e,,0,3698,e,,0
19181,16077,GESW,ERWERBST13,ELGPLUS,2018,171,e,,0,3696,e,,0
19182,16077,GESW,ERWERBST13,ELGPLUS,2019,166,e,,0,3496,e,,0
19183,16077,GESW,ERWERBST13,ELGPLUS,2020,167,e,,0,3600,e,,0


In [7]:
c.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19185 entries, 0 to 19184
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   KREISE                   19185 non-null  object
 1   GES                      19185 non-null  object
 2   ERW122                   19185 non-null  object
 3   ELGAT2                   19185 non-null  object
 4   JAHR                     19185 non-null  object
 5   ELG002_WERT              19185 non-null  int32 
 6   ELG002_QUALITAET         19185 non-null  object
 7   ELG002_GESPERRT          19185 non-null  object
 8   ELG002_WERT-VERFAELSCHT  19185 non-null  object
 9   ELG003_WERT              19185 non-null  int32 
 10  ELG003_QUALITAET         19185 non-null  object
 11  ELG003_GESPERRT          19185 non-null  object
 12  ELG003_WERT-VERFAELSCHT  19185 non-null  object
dtypes: int32(2), object(11)
memory usage: 1.8+ MB


In [8]:
from pprint import pprint 

pprint(c.metadata)

{'Copyright': '© Statistisches Bundesamt (Destatis), 2022',
 'Ident': {'Method': 'cube', 'Service': 'metadata'},
 'Object': {'Code': '22922KJ114',
            'Content': 'Statistik zum Elterngeld, Durchschnittl.Höhe des '
                       'monatl. Elterngeldanspruchs, Durchschnittliche Höhe '
                       'des Elterngeldanspruchs, Kreise, Geschlecht, '
                       'Erwerbstätigkeit, Art der Inanspruchnahme, Jahr',
            'State': 'undefiniert',
            'Statistic': {'Code': '22922',
                          'Content': 'Statistik zum Elterngeld',
                          'Updated': '25.03.2021 08:00:04h'},
            'Structure': {'Axis': [{'Code': 'KREISE',
                                    'Content': 'Kreise',
                                    'Type': 'klassifizierend',
                                    'Updated': '13.02.2019 16:42:01h'},
                                   {'Code': 'GES',
                                    'Content': 'Gesc